In [12]:
import xml.etree.ElementTree as ET
import Versaterm_Connect
import psycopg2
import traceback
import pandas as pd
import os

In [63]:
#GET field ucr_ext from Table ucr_ext from Versaterm
def sensitive_case(case_id):
    #print('Connecting to PostgreSQL database....')
    cnxn= None
    
    try: 
        # read connection parameters
        params = Versaterm_Connect.config_w_encryption()
        
        #Connection to DB
        cnxn = psycopg2.connect(**params)
    
        #print('Connection to PostgreSQL database SUCCESSFUL')

        #Cursor Object
        cursor = cnxn.cursor()
    
        query_text = "select concat (LTRIM(RTRIM( cast ( go_data.rucr as text ))) , LTRIM(RTRIM( cast ( go_data.rext as text ))) ) as rucr_ext FROM go_data WHERE go_data.primary_key= (%s)"
        
        #Testing query 
        cursor.execute(query_text, (case_id,))
        
        try:
            
            #Retrieve the records from the database
            records = cursor.fetchall()[0][0]

            sensitive_codes= ['9090', '54011', '549933', '549934', '54991'] 
            if(records in sensitive_codes):
                return True
            else:
                return False
        except:
            #print("Something wrong with case_id", case_id)
            return -1
    #Bad Credentials Exception
    except Exception as err:
        print(f"Connection to PostgreSQL FAILED! {err=}, {type(err)=}")
        
        #Where the exception happened
        print(traceback.format_exc())  # or: traceback.print_exc()
        raise
        
    #Close Connection
    finally:
        if cnxn is not None:
            #Close Session
            cursor.close()
            cnxn.close()
            #print('Connection to PostgreSQL database CLOSED')



In [52]:
def xml_to_pdf(file_path, pdf):
    
    first_id=1
    
    #Get DATA
    data=file_path

    #Build Tree
    tree = ET.parse(data)

    #Get Root
    root = tree.getroot()

    #Get Children of root
    children_root= list(root)

    
    text_dict = {'ActivityDescriptionText': '', 'ActivityDescriptionTextHtml': '', 'BinaryObject.Base64':''}

    def get_age(temp_parent):
        try:
            #Create a more robust method (look all children, with tag person age, and use that to calculate age-- no hard coding)
            age = int(temp_parent.find('{http://crash.dps.utah.gov/jxdm/1.0/extension}PersonAge').text)
            return age
        except:
            return -1
        
    #Recursive helper function
    def helper_function(temp_parent, temp_children, n):

        x=0

        for sub_child in temp_children:
            field= temp_parent.find(sub_child.tag)
            field_a = field.attrib if len(field.attrib) !=0 else ""
            
            if(sub_child.tag.split('}')[1] =="ID" and temp_parent.tag.split('}')[1]=="ActivityID"):
                global id_case
                id_case=sub_child.text
            
            indentation= "    "*n
            #Write to pdf  
            if(sub_child.tag.split('}')[1] == 'BinaryObject.Base64'):
                text_dict['BinaryObject.Base64'] = sub_child.text
                return
            elif(sub_child.tag.split('}')[1] == 'pin'):
                continue

            elif(temp_parent.tag.split('}')[1]=='InvolvedVehiclePassenger' and get_age(temp_parent)>=0 and get_age(temp_parent)<18):
                try:
                    pass_attribs= temp_children
                    sensitive_info = ['PersonName', 'Residence', 'PersonBirthDate','PersonPhysicalDetails','formname', 'PersonAge', 'pin']
                    for x in pass_attribs:
                            if(x.tag.split('}')[1] not in sensitive_info):
                                field= temp_parent.find(x.tag)
                                field_a = field.attrib if len(field.attrib) !=0 else ""
                                if(type(field_a)==dict):      
                                        pdf.cell(10, 10,indentation+x.tag.split('}')[1], 0, 0)

                                        if(len(field_a['Description']) >100):
                                            pdf.ln(1)
                                            pdf.multi_cell(0, 10,field_a['Description'] , align='L', border = 0,fill= False)
                                        else:
                                            pdf.cell(210, 10,indentation+field_a['Description'], 0 , 1 ,align='C')
                                else:
                                    helper_function(x, list(x), n+1)
                            else:
                                pdf.cell(10, 10,indentation+x.tag.split('}')[1], 0, 1)
                    return 
                    
                except:
                    print("Something wrong with person age parameter")
                    pass


            else:
                pdf.set_font('Times')
                try:
                    if(sub_child.text != None):
                        if(sub_child.text !="" and not sub_child.text.isspace()):
                            pdf.cell(10, 10,indentation+sub_child.tag.split('}')[1], 0, 0)
                except:
                    print("Problem with None Type  ", sub_child.tag.split('}')[1], sub_child.text)
                    pass

            if(type(field_a)==dict):
                if(len(field_a['Description']) >100):
                    pdf.ln(1)
                    pdf.multi_cell(0, 10,field_a['Description'] , align='L', border = 0,fill= False)
                else:
                    #pdf.multi_cell(160, 10,field_a['Description'] , align='R', border = 0,fill= False)
                    pdf.cell(210, 10,indentation+field_a['Description'], 0 , 1 ,align='C')

            else:

                try:
                    if(field.text!=None):
                        
                        if(len(field.text) >100):
                            pdf.ln(5)
                            pdf.multi_cell(0, 10,field.text , align='L', border = 0,fill= False)


                        else:
                            if(field.text!="" and not field.text.isspace()):                        
                                pdf.cell(210, 10,indentation+str(field.text)+" "+str(field_a), 0 , 1 ,align='C')

                except Exception:
                    print("Problem field_text", field)
                    pass


            if(len(list(sub_child))>0): 
                #Have some kind of recursion/ exhaustive loop (current system it just mere brute force :(  
                helper_function(sub_child, list(sub_child), n+1)

            x+=1
            
        return 

    #Get each element, associate with each child of root (i.e. reminder of tree)
    for child in children_root:

        pdf.set_font('Times', 'B', 13)
        pdf.cell(30, 10,"SECTION: ", 0, 0)
        pdf.cell(20, 10,child.tag.split('}')[1], 0, 1)
    
        temp_parent= child
        temp_children= list(child)

        x=0

        #No sub_element associated with current section (usually, submission status) 
        if(len(temp_children)==0):
            field_ab = temp_parent.attrib if len(temp_parent.attrib) !=0 else ""
            pdf.set_font('Times')
            #Don't print if empty
            if(temp_parent.text !="" and not temp_parent.text.isspace()):
                pdf.cell(10, 10,"    "+temp_parent.tag.split('}')[1], 0, 0)
                pdf.cell(200, 10,temp_parent.text, 0,0, align ='C')
                pdf.cell(90, 10,field_ab, 0,1)


        #Loop over each element of the temp_parent
        for sub_child in temp_children:


            try:
                
                if(sub_child.text!=None):
                    
                    field = child.find(sub_child.tag)
                    field_a = field.attrib if len(field.attrib) !=0 else ""
                    if((sub_child.text!="" and not sub_child.text.isspace()) or len(list(sub_child))>0):
                        pdf.set_text_color(r=0, g=0, b=0)
                        pdf.set_font('Times', 'B', 13)
                        if(sub_child.tag.split('}')[1] == 'ActivityDescriptionText'):
                            text_dict['ActivityDescriptionText'] = sub_child.text
                        elif(sub_child.tag.split('}')[1] == 'ActivityDescriptionTextHtml'):
                            text_dict['ActivityDescriptionTextHtml'] = sub_child.text
                        elif(sub_child.tag.split('}')[1] == 'pin'):
                            continue
                        else:
                            if(len(list(sub_child))>0):
                                pdf.cell(10, 10,sub_child.tag.split('}')[1], 0, 1)
                            else:
                                pdf.cell(10, 10,sub_child.tag.split('}')[1], 0, 0)

            except:
                print("Problem in sub_child")
                pass

            pdf.set_font('Times')
            if(type(field_a)==dict):
                if(len(field_a['Description']) >100):
                    pdf.ln(1)
                    pdf.multi_cell(0, 10,field_a['Description'] , align='L', border = 0,fill= False)
                else:
                    pdf.cell(210, 10,field_a['Description'], 0 , 1 ,align='C')

            else:
                try:
                    
                    if(field.text!=None):
                        
                        if(len(field.text) >100):

                            if(sub_child.tag.split('}')[1] != 'ActivityDescriptionText' and 
                               sub_child.tag.split('}')[1] != 'ActivityDescriptionTextHtml'):
                                pdf.ln(5)
                                pdf.multi_cell(0, 10,field.text , align='L', border = 0,fill= False)
                        else:
                            if(field.text!="" and not field.text.isspace()):
                                pdf.cell(210, 10,str(field.text)+" "+str(field_a) , 0 , 1 ,align='C')

                except Exception:
                    print("Problem in field.text") 
                    pass

            #Sub_sub_elements
            if(len(list(sub_child))>0): 
                #Have some kind of recursion/ exhaustive loop (current system it just mere brute force :( 
                helper_function(sub_child, list(sub_child), 1)

            x+=1
        pdf.ln(1)


    def remove_empty_lines(text):
        
        lines = text.split('\n')
        non_empty_lines = [line for line in lines if line.strip() != '']
        return '\n'.join(non_empty_lines)
    
    #Accident Diagram
    if(text_dict['BinaryObject.Base64']!=""):
        import base64
        data = text_dict['BinaryObject.Base64']
        imgdata = base64.b64decode(data)
        filename = 'traffic_image.jpeg'  # I assume you have a way of picking unique filenames

        with open(filename, 'wb') as f:
                f.write(imgdata)

        pdf.add_page()
        pdf.set_font('Times', 'B', 13)     
        pdf.image('traffic_image.jpeg',  5, 10, 200)
        pdf.cell(30, 10, 'Accident Diagram', 0, 0, 'C')

    #Narrative Text
    if(text_dict['ActivityDescriptionText'] !=""):
        #####REMOVE EXTRA LINES FROM DESCRIPTION (CODE BELOW-- INSERT HERE)
        pdf.add_page()
        pdf.set_font('Times', 'B', 13)     
        pdf.cell(30, 10, 'Narrative Text', 0, 1, 'C')
        pdf.set_font('Times')     
        pdf.multi_cell(0, 10,remove_empty_lines(text_dict['ActivityDescriptionText']) , align='L')


    #End of hard copy
    pdf.add_page()
    pdf.set_font('Times', 'B', 15)     
    pdf.cell(200, 160, '*** END OF HARDCOPY ***', 0, 1, 'C')

    #Output
    clean_case_id = re.sub('[^A-Za-z0-9]+', '', id_case)
    f_name='{clean_case_id}.pdf'.format(clean_case_id = clean_case_id)
    pdf.output(f_name, 'F')

In [2]:
#https://realpython.com/python-send-email/
def email(clean_case_id):
    
    import email, smtplib, ssl
    from email import encoders
    from email.mime.base import MIMEBase
    from email.mime.multipart import MIMEMultipart
    from email.mime.text import MIMEText

    subject = "TESTING - XML AUDIT ALERT SYSTEM"
    body = 'Hello Fellow Recorders (?), \n\nThis is a [TEST] alert being generated from the XML_to_PDF roduction script signifing a potential update/change/error. \n\nPlease, take a look at case {clean_case_id} more detailed explanation. \n\nBest, \nPython'.format(clean_case_id=clean_case_id)
    sender_email = "noreply_crime_intel@slcgov.com"
    receiver_email = ['martin.zanaj@slcgov.com',]

    # Create a multipart message and set headers
    message = MIMEMultipart()
    message["From"] = sender_email
    message["To"] = ", ".join(receiver_email)
    message["Subject"] = subject
    #message["Bcc"] = receiver_email  # Recommended for mass emails

    # Add body to email
    message.attach(MIMEText(body, "plain"))
    text = message.as_string()

    # Log in to server using secure context and send email
    with smtplib.SMTP("webmail.slcgov.com") as server:
        server.sendmail(sender_email, receiver_email, text)
        print("Email Sent")

Email Sent


In [53]:
from fpdf import FPDF
class PDF(FPDF):    
    # Page footer
    def footer(self):
        #Position at 1.5 cm from bottom
        self.set_y(-15)
        # Arial italic 8
        self.set_font('Arial', 'I', 9)
        # Page number
        self.cell(0, 10, id_case + " "+'Page ' + str(self.page_no()) + '/{nb}', 0, 0, 'C')
     


In [ ]:
import re

directory = 'H:\\AccidentXML'
n_dir=0
n_files=0
n_jur=0
n_sens=0
n_non_sens=0
n_bad= 0
for subdir, dirs, files in os.walk(directory):

    for file in files:
        f=os.path.join(subdir, file)
        raw_path=r'%s' % f

        #Build Tree
        tree = ET.parse(raw_path)

        #Get Root
        root = tree.getroot()

        try:
            #Case_ID
            case_id=root[1][0][0].text
            jurisdiction= case_id[0:2]
            if(jurisdiction =="SL"):
                clean_case_id = re.sub('[^A-Za-z0-9]+', '', case_id)
                FLAG=sensitive_case(clean_case_id)
                #Connection to POSTGRES
                if(FLAG==False):
                    #Continue 
                    print('\033[92m'+"NON-SENSITIVE CASE",case_id, raw_path+'\033[0m')

                    #print("\t\t\t SAFE CASE",case_id, raw_path)
                    #Instantiation of inherited class
                    pdf = PDF()
                    pdf.alias_nb_pages()
                    pdf.add_page()

                    #Header
                    pdf.image('logo_pb.png', 10, 8, 33)
                    pdf.set_font('Times', 'BIU', 17)
                    pdf.cell(80)

                    # Calculate width of title and position
                    w = pdf.get_string_width('Utah EMVA Report') + 6
                    pdf.set_x((210 - w) / 2)

                    # Title
                    pdf.cell(w, 10, 'Utah EMVA Report', 0, 0, 'C')

                    # Line break
                    pdf.ln(30)
                    xml_to_pdf(raw_path,pdf)
                    
                    #Delete picture that is created after saving it in pdf
                    if os.path.isfile("traffic_image.jpeg"):
                        os.remove("traffic_image.jpeg")
                    n_non_sens+=1
                elif(FLAG==True):
                    print('\033[93m'+ "SENSITIVE CASE",case_id, raw_path + '\033[0m')
                    n_sens+=1

                else:
                    print('\033[91m' +"BAD CASE ID",case_id, raw_path +'\033[0m')
                    email(clean_case_id)
                    n_bad+= 1

            else:
                print('\033[94m' +"DIFFERENT JURISDICTION", case_id +'\033[0m')
                n_jur+=1
        except:
            print(raw_path)
            print('Error in reading path for directory/ file')
            pass
        n_files+=1
    n_dir+=1
print("# of Directories: ", n_dir, " # of files: ",  n_files, " # of Non Sensitive files: ", n_non_sens, " # of Sensitive files: ", n_sens, 
     " # of Bad Case ID files: ", n_bad, " # Different Jurisdiction files: ", n_jur)